In [1]:
class Datenbank:
    shared_locks = dict()
    exclusive_locks = dict()

    def read(self, transaktion, objekt):
        status = False
        if (
            objekt in self.shared_locks
            and transaktion in self.shared_locks[objekt]
            or objekt in self.exclusive_locks
            and self.exclusive_locks[objekt] == transaktion
        ):
            print(f"Success: Transaction {transaktion} has read object {objekt}!")
            status = True
        else:
            print(
                f"Error: Transaction {transaktion} does not have a lock on object {objekt}!"
            )
            status = False

        return status

    def write(self, transaktion, objekt):
        status = False
        if (
            objekt in self.exclusive_locks
            and self.exclusive_locks[objekt] == transaktion
        ):
            print(f"Success: Transaction {transaktion} has written object {objekt}!")
            status = True
        else:
            print(
                f"Error: Transaction {transaktion} does not have a write lock on object {objekt}!"
            )
            status = False

        return status

    def sl(self, transaktion, objekt):
        status = False
        if objekt in self.exclusive_locks:
            status = False
        elif objekt in self.shared_locks:
            self.shared_locks[objekt].add(transaktion)
            status = True
        else:
            self.shared_locks[objekt] = set([transaktion])
            status = True

        if status:
            print(
                f"Success: Transaction {transaktion} has shared lock on object {objekt}!"
            )
        else:
            print(
                f"Error: Transaction {transaktion} could not acquire shared lock on object {objekt}!"
            )

        return status

    def xl(self, transaktion, objekt):
        status = False
        if objekt in self.exclusive_locks:
            status = False
        elif objekt in self.shared_locks:
            status = False
        else:
            self.exclusive_locks[objekt] = transaktion
            status = True

        if status:
            print(
                f"Success: Transaction {transaktion} has exclusive lock on object {objekt}!"
            )
        else:
            print(
                f"Error: Transaction {transaktion} could not acquire exclusive lock on object {objekt}!"
            )

        return status

    def ul(self, transaktion, objekt):
        status = False
        success = f"Success: Transaction {transaktion} has unlocked object {objekt}!"
        failure = f"Error: Object {objekt} was not locked!"

        try:
            if objekt in self.exclusive_locks:
                if self.exclusive_locks[objekt] != transaktion:
                    raise KeyError
                else:
                    del self.exclusive_locks[objekt]
                    print(success)
                    status = True
            elif objekt in self.shared_locks:
                if (
                    transaktion in self.shared_locks[objekt]
                    and len(self.shared_locks[objekt]) == 1
                ):
                    del self.shared_locks[objekt]
                else:
                    self.shared_locks[objekt].remove(transaktion)

                print(success)
                status = True
            else:
                print(failure)
                status = False
        except KeyError:
            print(
                f"Error: Transaction {transaktion} does not have a lock on object {objekt}!"
            )
            status = False

        return status

    def __init__(self):
        return

    def __del__(self):
        if len(self.shared_locks) != 0 or len(self.exclusive_locks) != 0:
            print("Error: Not all locks have been released!")
        else:
            print("Success: All locks have been released!")
        return

# Task

1. Design a 2PL-compliant schedule

2. Design an illegal schedule

3. Design a schedule with two transactions that leads to a deadlock.

In [2]:
db = Datenbank()

db.write("T0", "A")

db.read("T1", "B")

Error: Transaction T0 does not have a write lock on object A!
Error: Transaction T1 does not have a lock on object B!


False